In [2]:
import opensim as osim
import os
from simFunctions import *
from subprocess import call
from stan_utils import subject_specific_isometric_force
from stan_utils import *
import pandas as pd
from ezc3d import c3d

# Export C3D markers, ground reaction forces and EMG

## Folder Iterator Class

In [10]:
class c3d_folder_iterator:
    def __init__(self, list_of_folders, working_dir, session_dir, import_img = False, plot=False):
        self.list = list_of_folders
        self.input = working_dir + '/' + session_dir
        self.output = working_dir + '/' + session_dir
        self.img = import_img
        self.plot = plot

    def apply(self):   
        for folder in self.list:  
            task_file = folder + '.c3d'  
            input_dir = os.path.abspath(os.path.join(self.input, folder))
            output_dir = os.path.abspath(os.path.join(self.output, folder))

            # define order for ground reaction wrench
            labels_wrench = ['ground_force_vx', 'ground_force_vy', 'ground_force_vz',
                             'ground_force_px', 'ground_force_py', 'ground_force_pz',
                             'ground_torque_x', 'ground_torque_y', 'ground_torque_z']
            # the number of plates and the order of feet hitting them should be known in advance. Use Mokka to visualise C3D action.
            # in this case, 5 plates are active, feet order is [right, left, right, left, right]
            labels_force = ['plate1_' + label for label in labels_wrench] +\
                        ['plate2_'  + label for label in labels_wrench] +\
                        ['plate3_' + label for label in labels_wrench] +\
                        ['plate4_'  + label for label in labels_wrench] +\
                        ['plate5_' + label for label in labels_wrench] 
            
            # OpenSim data adapters
            adapter = osim.C3DFileAdapter()
            adapter.setLocationForForceExpression(osim.C3DFileAdapter.ForceLocation_CenterOfPressure)
            trc_adapter = osim.TRCFileAdapter()

            # extract data for task (e.g., walk, run)
            # get markers and forces
            print(os.path.join(input_dir, task_file))
            task = adapter.read(os.path.join(input_dir, task_file))
            markers_task = adapter.getMarkersTable(task)
            forces_task = adapter.getForcesTable(task)

            # process markers of task and save to .trc file
            rotate_data_table(markers_task, [1, 0, 0], -90)
            rotate_data_table(markers_task, [0, 1, 0], -90)
            trc_adapter = osim.TRCFileAdapter()
            trc_adapter.write(markers_task, os.path.join(output_dir, 'task.trc'))

            # process forces -- the rotation is lab-specific
            rotate_data_table(forces_task, [1, 0, 0], -90)
            rotate_data_table(forces_task, [0, 1, 0], -90)

            # conversion of unit (f -> N, p -> mm, tau -> Nmm)
            mm_to_m(forces_task, 'p1')
            mm_to_m(forces_task, 'p2')
            mm_to_m(forces_task, 'p3')
            mm_to_m(forces_task, 'p4')
            mm_to_m(forces_task, 'p5')

            mm_to_m(forces_task, 'm1')
            mm_to_m(forces_task, 'm2')
            mm_to_m(forces_task, 'm3')
            mm_to_m(forces_task, 'm4')
            mm_to_m(forces_task, 'm5')

            # refine ground reaction forces
            refine_ground_reaction_wrench(forces_task, ['f1', 'p1', 'm1'],
                                          stance_threshold=50, tau=0.001, debug=False)
            refine_ground_reaction_wrench(forces_task, ['f2', 'p2', 'm2'],
                                          stance_threshold=50, tau=0.001, debug=False)
            refine_ground_reaction_wrench(forces_task, ['f3', 'p3', 'm3'],
                                          stance_threshold=50, tau=0.001, debug=False)
            refine_ground_reaction_wrench(forces_task, ['f4', 'p4', 'm4'],
                                          stance_threshold=50, tau=0.001, debug=False)
            refine_ground_reaction_wrench(forces_task, ['f5', 'p5', 'm5'],
                                          stance_threshold=50, tau=0.001, debug=False)
            
            # export forces
            time = forces_task.getIndependentColumn()
            forces_task = forces_task.flatten(['x', 'y', 'z'])
            force_sto = create_opensim_storage(time, forces_task.getMatrix(), labels_force)
            force_sto.setName('GRF')
            force_sto.printResult(force_sto, 'task_grf', output_dir, 0.01, '.mot')

            # import emgs
            if self.img == True:
                importEMG(input_dir, task_file, output_dir)

            # plot
            if self.plot == True:
                plot_sto_file(os.path.join(output_dir, 'task.trc'),
                            os.path.join(output_dir, 'task.pdf'), 3)
                plot_sto_file(os.path.join(output_dir, 'task_grf.mot'),
                            os.path.join(output_dir, 'task_grf.pdf'), 3)
            



## Parent directory
Check the parent directory: working directory and session directory (if present) will have to be relative to it.

In [4]:
os.getcwd()

'd:\\Docs_ASUS\\WORK\\hip\\MotionStudy\\MotionData\\001'

### Static

In [5]:
working_dir = 'P01/Python'
session_dir = 'static' 
trial_names = ['Static03'] # static may have only one file or may have more

In [6]:
import shutil

for name in trial_names:
    path = os.path.join(working_dir, session_dir, name)
    if not os.path.exists(path):
        os.makedirs(path)
        filename = name + '.c3d'
        shutil.move(os.path.join(working_dir, session_dir) + '/' + filename , path + '/' + filename)

In [12]:
list_of_folders = trial_names
walking_iterator = c3d_folder_iterator(list_of_folders, working_dir, session_dir)
walking_iterator.apply()

d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\static\Static03\Static03.c3d


### Walking 

In [23]:
working_dir = 'P01/Python'
session_dir = 'walking' 
trial_names = ['Walking16', 'Walking17', 'Walking18', 'Walking19', 'Walking20', 'Walking21']

In [24]:
import shutil

for name in trial_names:
    path = os.path.join(working_dir, session_dir, name)
    if not os.path.exists(path):
        os.makedirs(path)
        filename = name + '.c3d'
        shutil.move(os.path.join(working_dir, session_dir) + '/' + filename , path + '/' + filename)

In [26]:
list_of_folders = trial_names
walking_iterator = c3d_folder_iterator(list_of_folders, working_dir, session_dir, import_img = True, plot=True)
walking_iterator.apply()

d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\walking\Walking16\Walking16.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\walking\Walking17\Walking17.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\walking\Walking18\Walking18.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\walking\Walking19\Walking19.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\walking\Walking20\Walking20.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\walking\Walking21\Walking21.c3d


### Running

In [32]:
working_dir = 'P01/Python'
session_dir = 'running' 
trial_names = ['Running01', 'Running03',  'Running04', 'Running06', 'Running07', 'Running08', 'Running15']

In [33]:
import shutil

for name in trial_names:
    path = os.path.join(working_dir, session_dir, name)
    if not os.path.exists(path):
        print(path)
        os.makedirs(path)
        filename = name + '.c3d'
        shutil.move(os.path.join(working_dir, session_dir) + '/' + filename , path + '/' + filename)

P01/Python\running\Running01
P01/Python\running\Running03
P01/Python\running\Running04
P01/Python\running\Running06
P01/Python\running\Running07
P01/Python\running\Running08
P01/Python\running\Running15


In [34]:
running_iterator = c3d_folder_iterator(list_of_folders, working_dir, session_dir, import_img = True, plot=True)
running_iterator.apply()

d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running01\Running01.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running03\Running03.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running04\Running04.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running06\Running06.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running07\Running07.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running08\Running08.c3d
d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\running\Running15\Running15.c3d
